# Bragg-edge imaging with ODIN

This notebook illustrates how to convert recorded events on the ODIN detector to a single wavelength spectrum,
revealing a Bragg edge in the data.
WFM mode was used in the chopper cascade.

## Loading dataset

> Loader is not part of ``essimaging`` since McStas dataset format is not stabilized yet.

In [ ]:
import scipp as sc

from ess.imaging.data import get_mcstas_ob_images_path, get_mcstas_sample_images_path
from odin_mcstas_helper import (
    load_odin_simulation_data,
    McStasManualResolution,
    PLAIN_GRAPH,
)
from ess.reduce.nexus.types import FilePath


example_resolution = McStasManualResolution((128, 128))
# Small resolution for faster testing and documentation build.
ob_file_path = FilePath(get_mcstas_ob_images_path())
sample_file_path = FilePath(get_mcstas_sample_images_path())
ob_da = load_odin_simulation_data(ob_file_path, resolution=example_resolution)
sample_da = load_odin_simulation_data(sample_file_path, resolution=example_resolution)
sample_da

## Convert McStas raw data to NeXus

The raw McStas data looks different from what data in a NeXus file would look like.
The time-of-flight recorded by the McStas monitor is a unwrapped time of arrival
(see https://scipp.github.io/scippneutron/user-guide/chopper/frame-unwrapping.html);
the `tof` coordinate has values beyond 71ms,
as can be seen in the plot above.

The workflow that computes wavelengths from the WFM chopper cascade expects data in the NeXus format,
so we transform the data here.

In [ ]:
from odin_mcstas_helper import to_nexus

sample_nexus = to_nexus(sample_da, resolution=example_resolution)
ob_nexus = to_nexus(ob_da, resolution=example_resolution)

sample_nexus

In [ ]:
# Visualize
fig_sample = (
    sample_nexus.bins.concat()
    .hist(event_time_offset=300)
    .plot(title='McStas simulation: sample\n(wrapped like ESS format, 14Hz pulse)')
)
fig_ob = (
    ob_nexus.bins.concat()
    .hist(event_time_offset=300)
    .plot(title='McStas simulation: open beam\n(wrapped like ESS format, 14Hz pulse)')
)
fig_sample + fig_ob

In [ ]:
import plopp as pp

pp.scatter3d(sample_nexus.hist(), pos='position', cbar=True, pixel_size=0.0005)

## Choppers

To accurately compute the wavelengths of the neutrons from their time-of-arrival,
we need the parameters of the choppers in the beamline.

In [ ]:
import sciline as sl
from odin_mcstas_helper import DISK_CHOPPERS

disk_choppers = DISK_CHOPPERS.copy()  # Copy to avoid modifying the original dictionary

In [ ]:
disk_choppers["WFMC_1"]

### Check that the chopper settings make sense with a quick `tof` run

As useful sanity check is to run a basic simulation,
propagating neutrons through the chopper cascade,
using the [Tof](https://tof.readthedocs.io) package.

In [ ]:
from ess.reduce.time_of_flight.fakes import FakeBeamline

Ltotal = sample_nexus.coords['Ltotal'].mean()
ess_beamline = FakeBeamline(
    choppers=disk_choppers,
    monitors={"detector": Ltotal},
    run_length=sc.scalar(1 / 14, unit="s") * 8,
    events_per_pulse=100_000,
)

ess_beamline.model_result.plot()

We observe that the WFM choppers make 6 distinct frames at the detector,
and that the other choppers skip every other pulse to maximize wavelength coverage.

We can now compare the counts on the detector to our raw data,
to make sure they broadly resemble each other.

In [ ]:
raw_data = ess_beamline.get_monitor("detector")[0]

# Visualize
fig_ob + raw_data.hist(event_time_offset=300).squeeze().plot(
    title='Tof simulation (no sample)'
)

## Use WFM workflow

We now set up the workflow which will convert the raw neutron arrival times to a real time-of-flight,
and thus a wavelength.

### Setting up the workflow

In [ ]:
from ess.reduce import time_of_flight

workflow = sl.Pipeline(
    time_of_flight.providers(), params=time_of_flight.default_parameters()
)
workflow[time_of_flight.RawData] = sample_nexus
workflow[time_of_flight.Ltotal] = sample_nexus.coords['Ltotal']
workflow[time_of_flight.LtotalRange] = (
    sc.scalar(55.0, unit="m"),
    sc.scalar(65.0, unit="m"),
)
workflow[time_of_flight.PulseStride] = 2  # Need for pulse-skipping
workflow[time_of_flight.SimulationResults] = time_of_flight.simulate_beamline(
    choppers=disk_choppers, neutrons=2_000_000
)

workflow.visualize(time_of_flight.TofData)

### Inspect the lookup table

In [ ]:
table = workflow.compute(time_of_flight.TimeOfFlightLookupTable).squeeze()

table['pulse', 0].plot(title='Pulse 0') + table['pulse', 1].plot(title='Pulse 1')

### Compute neutron time-of-flight

In [ ]:
sample_tofs = workflow.compute(time_of_flight.TofData)
sample_tofs

In [ ]:
sample_wavs = sample_tofs.transform_coords('wavelength', graph=PLAIN_GRAPH)
sample_wavs

We can now compare our computed wavelengths to the true wavelengths of the neutrons in the McStas simulation:

In [ ]:
true_wavs = sample_da.hist(sim_wavelength=300).rename(sim_wavelength='wavelength')

pp.plot(
    {
        'true': true_wavs,
        'wfm': sample_wavs.bins.concat().hist(
            wavelength=true_wavs.coords['wavelength']
        ),
    },
    title="ODIN McStas simulation",
)

## Region of interest

Looking at the counts on the 2d detector panel,
we see that there is a central rectangular darker region,
surrounded by brighter edges.

In [ ]:
sample_folded = sample_wavs.fold(dim='pixel_id', sizes={'y': 128, 'x': 128})
sample_folded.hist().plot(aspect='equal')

The dark region is where the beam was absorbed by the sample,
and this is the region of interest.
The brighter edges need to be discarded.

We crop the data using simple array slicing:

In [ ]:
sel = slice(11, 116, 1)
sample_cropped = sample_folded['y', sel]['x', sel]
sample_cropped.hist().plot(aspect='equal')

### Repeat for the open-beam

We repeat the conversion to wavelength and crop the edges of the open-beam measurement.

In [ ]:
# Give the same pixel positions to both sample and open beam.
# Note: this is only because of the way we computed the positions.
# In practice, the geometry should come from the nexus file and this won't be needed.
ob_nexus.coords.update(
    {key: sample_nexus.coords[key] for key in ('position', 'Ltotal')}
)

workflow[time_of_flight.Ltotal] = ob_nexus.coords['Ltotal']
workflow[time_of_flight.RawData] = ob_nexus

ob_tofs = workflow.compute(time_of_flight.TofData)
ob_wavs = ob_tofs.transform_coords('wavelength', graph=PLAIN_GRAPH)
ob_folded = ob_wavs.fold(dim='pixel_id', sizes={'y': 128, 'x': 128})
ob_cropped = ob_folded['y', sel]['x', sel]

## Normalize the signal

Finally, we are able to normalize our sample measurement to the open-beam data.

Here, we sum over all pixels before normalizing.
There is no spatial structure in the signal, and we are only interested in the wavelength spectrum (where the Bragg edge is).
So this is effectively like degrading the detector resolution to a single pixel.

In [ ]:
# Common set of bins
bins = sc.linspace('wavelength', 1.1, 9.5, 301, unit='angstrom')

num = sample_cropped.bins.concat().hist(wavelength=bins)
den = ob_cropped.bins.concat().hist(wavelength=bins)

# Add variances
num.variances = num.values
den.variances = den.values

normalized = num / den
normalized

In [ ]:
normalized.plot()

## Save the final result

In [ ]:
from scippneutron.io import save_xye

to_disk = normalized.copy(deep=False)
to_disk.coords['wavelength'] = sc.midpoints(to_disk.coords['wavelength'])

save_xye('fe_bragg_edge.xye', to_disk)